In [1]:
using SpecialFunctions, IterTools, HardSphereDynamics, StatsBase # For Std Error.

Aqui abajo Rosa define todos los resultados téoricos para cualquier número de dimensiones, siempre
que no haya "sombras prohibidas" (es decir, que las bolas siempre quepan unas al lado de otras).

In [2]:
function radiusmax(hs::Array)
    #Maximal Radius for two spheres in box of sizes hs
    n=length(hs)
    a=4*(n-1)
    b=4*sum(hs)
    c=sum(hs.^2)
    result=(b-sqrt(b^2-4*a*c))/(2*a)
end

function SphereArea(n)
    return 2*(pi^(n/2))/gamma(n/2)
end

function NormalizationFactor(n, balls=2)
    d = 2*n
    return SphereArea(d)/SphereArea(d-1)*(d-1)
end

function Ak(boxdims,m)
    result = 0
    for aks in subsets(boxdims, m)
        result += prod(aks)
    end
    return result
end

function AvailableVolume(n, boxdims, r)
    vol = prod((2*boxdims).^2)
    for k in 0:n
        vol -= (4^n)*(pi)^(n/2-k/2)*r^(k+n)/gamma(k/2+n/2+1)*Ak(boxdims, n-k)*(-1)^k
    end
    return vol
end

function SphereCollisionArea(n, boxdims, r)
    area = 0 
    for k in 0:n
        area += 2^(2*n+1/2)*(pi^(n/2-k/2))*(r^(k+n-1))/gamma(k/2+n/2)*Ak(boxdims, n-k)*(-1)^k
    end
    return area
end

function WallCollisionArea(n, boxdims, r, dim)
    otherboxdims = deleteat!(copy(boxdims),dim)
    area = prod((2*otherboxdims).^2)*(2*boxdims[dim])
    for k in 0:(n-1)
        area -= 2^(2*n-2)*(pi^(n/2 - k/2))*(r^(k+n))/gamma(k/2+n/2+1)*(-1)^k*Ak(otherboxdims, n-1-k)
    end
    return area
end

function SphereCollisionTime(n, boxdimsh, r)
    boxdimsa = boxdimsh/2 .- r 
    return NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/SphereCollisionArea(n, boxdimsa, r)
end

function WallCollisionTime(n, boxdimsh, r, dim)
    boxdimsa = boxdimsh/2 .- r 
    return NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/WallCollisionArea(n, boxdimsa, r, dim)
end
    
function HoppingCollisionTime(n, boxdimsh, r, dim)
    boxdimsa = boxdimsh/2 .- r 
    otherboxdims = deleteat!(copy(boxdimsa),dim)
    #if r < minimum(boxdimsh)/4
        result= NormalizationFactor(n)*AvailableVolume(n, boxdimsa, r)/
           AvailableVolume(n-1, otherboxdims, r)/2^(2.5)/boxdimsa[dim]
    #else
    #    result=1.0
    #end
    
end

function CollisionTimesTheo(r, n=3, boxdimsh = [1.0, 1.5, 1.75])
    wall_times = []
    hop_times = []
    for dim in 1:n
        w = WallCollisionTime(n, boxdimsh, r, dim)
        push!(wall_times, w, w)
        push!(hop_times, HoppingCollisionTime(n, boxdimsh, r, dim))
    end
    result = hcat(r , SphereCollisionTime(n, boxdimsh, r), wall_times..., hop_times...)
    return result
end



CollisionTimesTheo (generic function with 3 methods)

In [28]:
?mod

search: mod modf mod1 mode modes module mod2pi Module modelmatrix model_response



```
mod(x::Integer, r::AbstractUnitRange)
```

Find `y` in the range `r` such that $x ≡ y (mod n)$, where `n = length(r)`, i.e. `y = mod(x - first(r), n) + first(r)`.

See also: [`mod1`](@ref).

# Examples

```jldoctest
julia> mod(0, Base.OneTo(3))
3

julia> mod(3, 0:2)
0
```

!!! compat "Julia 1.3"
    This method requires at least Julia 1.3.


---

```
mod(x, y)
rem(x, y, RoundDown)
```

The reduction of `x` modulo `y`, or equivalently, the remainder of `x` after floored division by `y`, i.e. `x - y*fld(x,y)` if computed without intermediate rounding.

The result will have the same sign as `y`, and magnitude less than `abs(y)` (with some exceptions, see note below).

!!! note
    When used with floating point values, the exact result may not be representable by the type, and so rounding error may occur. In particular, if the exact result is very close to `y`, then it may be rounded to `y`.


```jldoctest
julia> mod(8, 3)
2

julia> mod(9, 3)
0

julia> mod(8.9, 3)
2.9000000000000004

julia> mod(eps(), 3)
2.220446049250313e-16

julia> mod(-eps(), 3)
3.0
```

---

```
rem(x::Integer, T::Type{<:Integer}) -> T
mod(x::Integer, T::Type{<:Integer}) -> T
%(x::Integer, T::Type{<:Integer}) -> T
```

Find `y::T` such that `x` ≡ `y` (mod n), where n is the number of integers representable in `T`, and `y` is an integer in `[typemin(T),typemax(T)]`. If `T` can represent any integer (e.g. `T == BigInt`), then this operation corresponds to a conversion to `T`.

# Examples

```jldoctest
julia> 129 % Int8
-127
```


In [44]:
using Parameters, Statistics, HardSphereDynamics, StaticArrays

function BallsRelativeSign(state)
    b1, b2 = state
    return broadcast(sign, b1.x - b2.x)
end

function mean_interval(times)
    return mean(diff(times))
end

h1 = 1.0
h2 = 1.75
h3 = 1.33
h4=1.40
r = 0.1
rmax=  radiusmax([h1,h3,h4])   #maximal possible radius hopping in h2 direction

Nmax=10^7

function CollisionTimesSim4D(r, h1 = h1, h2 = h2, 
                    h3 = h3, h4 = h4 ,num_collisions = Nmax)
    table = HardSphereDynamics.RectangularBox(SA[-h1/2, -h2/2, -h3/2, -h4/2],
                                              SA[+h1/2, +h2/2, +h3/2, +h4/2])

    n = 4
    spheres = 2
    fluid = HardSphereFluid{n,Float64}(table, spheres, r)
    initial_condition!(fluid.balls, table)

    # set up simulation:
    collision_type = ElasticCollision()
    flow_type = ExternalFieldFlow(SA[0.0, 0.0, 0.0, 0.0])
    event_handler = AllToAll(fluid, flow_type)

    simulation =  HardSphereSimulation(
        fluid, event_handler, flow_type, collision_type);

    @unpack fluid, flow_dynamics, collision_dynamics, event_handler = simulation
    states = [deepcopy(fluid.balls)]
    times = [0.0]
    collision_types = [:none]
    partner2 = [0] 
    partner1 = [0]
    hopping_times = []
    hopping_directions = []
    for i in 1:num_collisions
        HardSphereDynamics.flow!(fluid.balls, (event_handler.next_collision_time - simulation.current_time), 
            flow_dynamics)
        simulation.current_time = event_handler.next_collision_time
        push!(collision_types, event_handler.collision_type)
        push!(partner2, event_handler.partner2)
        push!(partner1, event_handler.partner1)
        HardSphereDynamics.collide!(fluid, event_handler, collision_dynamics)
        HardSphereDynamics.find_collision!(event_handler, fluid, flow_dynamics)
        push!(states, deepcopy(fluid.balls))
        push!(times, simulation.current_time)
        prev, current = states[i:i+1]
        signprev, signcurrent = broadcast(BallsRelativeSign,[prev, current])
        if any(signprev .!= signcurrent)
            b1, b2 = states[i]
            start_time, final_time = times[i:i+1]
            hop_times = start_time .+( b1.x - b2.x)./(b2.v - b1.v)
            for (dir, time) in enumerate(hop_times)
                if all([time>=start_time, time<=final_time])
                    push!(hopping_times, time)
                    push!(hopping_directions, dir)
                end
            end
        end
    end
    
    wall_idx = (collision_types.== :wall_collision).&(partner1.== 1)
    wall_times = times[wall_idx];
    which_wall = partner2[wall_idx];
    wall_means = []
    
    # The std errros are no longer missin!
    wall_errors = []
    hop_errors  = []
    
    for i in 1:length(fluid.box.walls)
        
        enparedcorrecta=wall_times[which_wall .== i]
        error=sem(diff(enparedcorrecta))
        nn=length(enparedcorrecta)
     #   println( " En la pared $i, chocamos $nn veces")
        
        push!(wall_means,mean_interval(enparedcorrecta))
        push!(wall_errors, error)
        
    end
    
    hop_means = []
    hop_errors= []

    for i in 1:n
        
        cual_hop=hopping_times[hopping_directions .== i]
        ll=length(cual_hop)
        if ll > 2
            error=sem(diff(cual_hop))
            
            if mod(i,2) == 0
            println("en el hop direccion $i, hopamos $ll veces con radio $r")
            end
            
            push!(hop_means, mean_interval(cual_hop))
            push!(hop_errors, error)
        else
            push!(hop_means, NaN)
            push!(hop_errors, NaN)
        end
        
    end
    
    colision=times[collision_types.== :disc_collision]
    meandisc=mean_interval(colision)
    errorcol=sem(diff(colision))
    
    
    result=hcat(r, meandisc, errorcol,wall_means..., wall_errors..., hop_means..., hop_errors...)
    
    return result
        
end


CollisionTimesSim4D (generic function with 6 methods)

In [45]:
hs=[h1,h3]
radiusmax(hs), rmax

(0.34952467848499547, 0.4048223048109765)

In [47]:
@time chin, pun, cuaz, el, eslam = CollisionTimesSim4D(0.21)

en el hop direccion 2, hopamos 813894 veces con radio 0.21
en el hop direccion 4, hopamos 1211691 veces con radio 0.21
113.807821 seconds (231.68 M allocations: 15.221 GiB, 21.72% gc time)


1×27 Array{Float64,2}:
 0.21  5.45811  0.00682143  3.83204  …  0.00472213  0.0020913  0.0024459

In [48]:
deltaslog=-11:0.5:-3
rs=rmax.-exp.(deltaslog);

In [49]:
#theo = vcat(broadcast(CollisionTimesTheo, rs)...);
@time sim = vcat(broadcast(CollisionTimesSim4D, rs)...);

en el hop direccion 4, hopamos 395589 veces con radio 0.4048056031101862
en el hop direccion 4, hopamos 394348 veces con radio 0.4047947683616267
en el hop direccion 4, hopamos 396681 veces con radio 0.404776904881214
en el hop direccion 4, hopamos 395694 veces con radio 0.4047474529810888
en el hop direccion 4, hopamos 397396 veces con radio 0.40469889500688977
en el hop direccion 4, hopamos 396602 veces con radio 0.40461883644196583
en el hop direccion 4, hopamos 395799 veces con radio 0.40448684218307396
en el hop direccion 4, hopamos 395899 veces con radio 0.40426922044082864
en el hop direccion 4, hopamos 400102 veces con radio 0.40391042284542195
en el hop direccion 4, hopamos 405218 veces con radio 0.4033188656179989
en el hop direccion 4, hopamos 411205 veces con radio 0.4023435526343101
en el hop direccion 4, hopamos 420316 veces con radio 0.4007355333725124
en el hop direccion 2, hopamos 5 veces con radio 0.398084357811891
en el hop direccion 4, hopamos 436981 veces con radio

In [ ]:
using JLD

In [ ]:
nota=" En esta corrida solo guardamos resultados numericos.
Vienen en este orden.
Columna      Que
1            radios
2            choque bolas
3            error choque bolas
4            pared 1 positiva
5            error pared 1 positiva
6,7          igual pared 1 negativa
8,9,10,11          idem pared 2
12,13,14,15  igual pared 3 (4dimensiones)
16 a 19      igual pared 4
20,21        hopp 1, error hop 1
22,23        hopp 2, error hop 2
24 a 27  hops los otros obio
"

otranota="La variable h contiene los tamaños de la caja. 
La variable Nmax el numero de collisiones total.
La variable rmax es el radio limite para cruzar en la direccion h2 (hopping in y direction)"
onde="hoppingclosetozero4D-1e8-20210225.jld"

h=(h1,h2,h3, h4)

save(onde, "nota01", nota, "nota02", otranota, "numericas", sim, "h", h, "Nmax", Nmax, "rmax", rmax)


In [ ]:
sim[:,17]
rs

In [ ]:
using Plots

In [ ]:
paupau=plot(xlim=(0.1,rs[1]+0.1), ylim=(0.001,4),
   # xscale=:log,
   # yscale=:log,
    ylabel="mean horizontal hopping time", size=(450, 300))
scatter!(rs,sim[:,22], yerror=sim[:23])
title!("hopp along h2")
xlabel!("r")
ylabel!("mean time")

In [ ]:
sim

In [ ]:
scatter(sim[:,1], sim[:,10])
title!("Hopping y")
xlabel!("r")
ylabel!("mean time")

In [ ]:
savefig("hop_axis_2_3D_inset.pdf")

In [ ]:
scatter(rs,sim[:,18] )
plot!(rs,theo[:,11])
title!("Hopping z")
xlabel!("r")
ylabel!("mean time")

A box with different dimensions

In [ ]:
h1 = 1
h2 = 2
h3 = 3
theo2 = vcat(broadcast(function (r)  CollisionTimesTheo(r, 3, [h1,h2,h3]) end, rs)...);
sim2 = vcat(broadcast(function (r) CollisionTimesSim(r, h1, h2, h3) end , rs)...);

nota="Los Resultados Teoricos y Simulados estan en dos tablas.
Primera columna son los radios.
Prmero las bolas entre ellas.
Luego vienen en Teoricas los golpes con las paredes de un bola 
contra una pared asi: 
Columna   Pared
2         x positiva
3         x negativa
4         y positiva
5         y negativa
6         z positiva
7         z negativa
Luego vienen los hops, 
8         hop x
9         hop y
10        hop z
En las simulaciones, entre cada columna, en el mismo orden, hay una columna 
de errores estandar.
"
otranota="La variable h contiene los tamaños de la caja. La variable Nmax el numero de collisiones total."
onde="simulaRosa20201216-1930.jld"

h=(h1,h2,h3)

save(onde, "nota01", nota, "nota02", otranota, 
    "teoricas" , theo, "numericas", sim, "h", h, "Nmax", Nmax)

In [ ]:
using Plots
scatter(rs,[s[1] for s in sim2])
plot!(rs,[t[1] for t in theo2])
title!("Disc collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][1] for s in sim2])
plot!(rs,[t[2][1] for t in theo2])
title!("x wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][3] for s in sim2])
plot!(rs,[t[2][3] for t in theo2])
title!("y wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[2][5] for s in sim2])
plot!(rs,[t[2][5] for t in theo2])
title!("z wall collisions")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][1] for s in sim2])
plot!(rs,[t[3][1] for t in theo2])
title!("Hopping x")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][2] for s in sim])
plot!(rs,[t[3][2] for t in theo])
title!("Hopping y")
xlabel!("r")
ylabel!("mean time")

In [ ]:
scatter(rs,[s[3][3] for s in sim2])
plot!(rs,[t[3][3] for t in theo2])
title!("Hopping z")
xlabel!("r")
ylabel!("mean time")